In [1]:
from ultralytics import YOLO
import torch
from PIL import Image
import cv2
import time
import sys
import os
from datetime import datetime

C:\Users\Aizat\anaconda3\envs\YoloV8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [303]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\Aizat/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-5 Python-3.9.16 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
#Custom Model

model = torch.hub.load('ultralytics/yolov5', 'custom', path='konlovephone.pt')
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='gesture3.pt')
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='best4.pt')

Using cache found in C:\Users\Aizat/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-5 Python-3.9.16 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [7]:
confi_threshold = 0.20 #threshold value for confidence score
gesturePresent = False;
startTime = 0;
elapseTime = 0;
delayDetection = 3;
scanning_class = -1;

display_msg = '';

masterPassword = [1,1,0];
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
print(masterPassword)


[1, 1, 0]


In [6]:
facePresent = False;
startTime_faceDet = 0;
elapseTime_faceDet = 0;
delayDetection_face = 6.0;

In [8]:
cap = cv2.VideoCapture(0)

cv2.namedWindow("img", cv2.WINDOW_NORMAL)
  
# Using resizeWindow()
cv2.resizeWindow("img", 900, 700)
password = [];



while True:
    
    #reads the frames
    _, img = cap.read()
    
    
    #convert to grayscaleq
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect the faces
    faces = face_classifier.detectMultiScale(gray, 1.1, 4)
    
    #If a new Face is Present
    if len(faces) == 1 and (startTime_faceDet == 0):
        facePresent = True;
        startTime_faceDet = time.time();
        
    if len(faces) == 1 and (startTime_faceDet > 0):
        elapseTime_faceDet = round((time.time() - startTime_faceDet), 2);
        if(elapseTime_faceDet > delayDetection_face):
            print("Face is Detected")
            break;
    
    pic = cv2.putText(img, str(elapseTime_faceDet), (5,60), cv2.FONT_HERSHEY_PLAIN, 5 ,(0, 0, 255));
    
    
    if (len(faces) == 0 or len(faces) > 1):
        facePresent = False;
        startTime_faceDet = 0;
        print('No Face Detected; Timer Reset');
        pic = cv2.putText(img, "No Face Detected", (5,60), cv2.FONT_HERSHEY_PLAIN, 5 ,(0, 0, 255));
    
    
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        pic = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Display
    cv2.imshow('img', pic)
    
    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release();
        cv2.destroyAllWindows();
        
        
while True:
    _, frames = cap.read()  #reads each frame
    results = model(frames) #has the model predict the frame and predicts for objects 
    
    #debug
    #print('---------------------')
    #print(results.pandas().xyxy[0])
    #print('Detected Gestures : ' + str(len(results.pandas().xyxy[0])))
    
    max_detected_confi = 0;
    
    #for each object detected 
    for i in range(0, len(results.pandas().xyxy[0])):
        confi = results.xyxy[0][i][4].numpy() #retrieves the confidence score
        #print(results.pandas().xyxy[0]);
        
        #takes the highest confidence score gesture. Instead of ensuring only 1 gesture is present. We just take the highest score present in the list
        if confi > max_detected_confi:
            max_detected_confi = confi
            x0,y0,x1,y1,confi,cla = results.xyxy[0][i].numpy() #retrieves the bounding box points, confidence score, class,
            
            origin = (x0.astype(int), y0.astype(int));
            end = (x1.astype(int), y1.astype(int));
            
            #label = cv2.rectangle(frames, origin,end, (255,0,0), 2 )
            
            label = cv2.rectangle(frames, origin, end, (255,0,0), 2 )
            label = cv2.putText(frames, str(cla), (x0.astype(int), y0.astype(int) + 10), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
            
            
        
    if confi > confi_threshold:
        
        #If a new gesture is Present
        if (startTime == 0):
            scanning_class = cla
            gesturePresent = True;
            startTime = time.time();
        
        #if same class as before, start time
        if (scanning_class == cla and startTime > 0):
            elapseTime = round((time.time() - startTime), 2)
            label = cv2.putText(frames, str(elapseTime), (5, 30), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
            #if passes 3 three seconds, that gesture is added to the bank
            #print(elapseTime)
            if(elapseTime > delayDetection):    
                display_msg = "Gesture Recognised - " + str(cla);
                password.append(int(cla));
                print(display_msg)
                startTime = 0;
                    
    
    if (scanning_class != cla or confi < confi_threshold):
        gesturePresent = False;
        startTime = 0;
        #print('Bad Input. No Gesture Recognised or Not Clear');
            
            
    if (len(password) > 2):
        if password == masterPassword:
            print(password);
            print("You have Entered the Correct Password");
        else:
            print(password);
            print("Wrong Password");
        break
    
    
    pass_text = ''.join(str(x) for x in password); 
    label = cv2.putText(frames, pass_text, (5, 500), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
    cv2.imshow('img', label)
    
    
    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release();
        cv2.destroyAllWindows();
        

        
        
        


        
cap.release();
cv2.destroyAllWindows();
time_enter = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

saveFile = cv2.putText(img, time_enter, (5, 60), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
filename = 'savedImage.jpg'
cv2.imwrite(filename, saveFile)








Face is Detected
Gesture Recognised - 1.0
Gesture Recognised - 1.0
Gesture Recognised - 1.0
[1, 1, 1]
Wrong Password


True

In [270]:
cap = cv2.VideoCapture(0)

cv2.namedWindow("img", cv2.WINDOW_NORMAL)
  
# Using resizeWindow()
cv2.resizeWindow("img", 900, 700)
password = [];


while True:
    
    #reads the frames
    _, img = cap.read()
    
    
    #convert to grayscaleq
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect the faces
    faces = face_classifier.detectMultiScale(gray, 1.1, 4)
    
    #If a new Face is Present
    if len(faces) == 1 and (startTime_faceDet == 0):
        facePresent = True;
        startTime_faceDet = time.time();
        
    if len(faces) == 1 and (startTime_faceDet > 0):
        elapseTime_faceDet = round((time.time() - startTime_faceDet), 2);
        if(elapseTime_faceDet > delayDetection):
            print("Face is Detected")
            break;
    
    pic = cv2.putText(img, str(elapseTime_faceDet), (5,60), cv2.FONT_HERSHEY_PLAIN, 5 ,(0, 0, 255));
    
    
    if (len(faces) == 0 or len(faces) > 1):
        facePresent = False;
        startTime_faceDet = 0;
        print('No Face Detected; Timer Reset');
        pic = cv2.putText(img, "No Face Detected", (5,60), cv2.FONT_HERSHEY_PLAIN, 5 ,(0, 0, 255));
    
    
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        pic = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Display
    cv2.imshow('img', pic)
    
    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
        
# Release the VideoCapture object
cap.release();
cv2.destroyAllWindows();

No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face Detected; Timer Reset
No Face De

In [286]:
cap = cv2.VideoCapture(0)
cv2.namedWindow("output", cv2.WINDOW_NORMAL)
  
# Using resizeWindow()
cv2.resizeWindow("output", 900, 700)

password = [];


while True:
    _, frames = cap.read()  #reads each frame
    results = model(frames) #has the model predict the frame and predicts for objects 
    
    #debug
    #print('---------------------')
    #print(results.pandas().xyxy[0])
    #print('Detected Gestures : ' + str(len(results.pandas().xyxy[0])))
    
    max_detected_confi = 0;
    
    #for each object detected 
    for i in range(0, len(results.pandas().xyxy[0])):
        #print(results.pandas().xyxy[0])
        confi = results.xyxy[0][i][4].numpy() #retrieves the confidence score
        print(results.pandas().xyxy[0]);
        #takes the highest confidence score gesture. Instead of ensuring only 1 gesture is present. We just take the highest score present in the list
        
        if confi > max_detected_confi:
            max_detected_confi = confi
            x0,y0,x1,y1,confi,cla = results.xyxy[0][i].numpy() #retrieves the bounding box points, confidence score, class,
            
            origin = (x0.astype(int), y0.astype(int));
            end = (x1.astype(int), y1.astype(int));
            
            #label = cv2.rectangle(frames, origin,end, (255,0,0), 2 )
            
            #label = cv2.rectangle(frames, origin, end, (255,0,0), 2 )
            label = cv2.putText(frames, str(cla), (x0.astype(int), y0.astype(int) + 30), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
            
            
        
    if confi > confi_threshold:
        
        #If a new gesture is Present
        if (startTime == 0):
            scanning_class = cla
            gesturePresent = True;
            startTime = time.time();
        
        #if same class as before, start time
        if (scanning_class == cla and startTime > 0):
            elapseTime = round((time.time() - startTime), 2)
            label = cv2.putText(frames, str(elapseTime), (5, 60), cv2.FONT_HERSHEY_PLAIN, 6 ,(0, 0, 255));
            #if passes 3 three seconds, that gesture is added to the bank
            #print(elapseTime)
            if(elapseTime > delayDetection):    
                display_msg = "Gesture Recognised - " + str(cla);
                password.append(int(cla));
                print(display_msg)
                startTime = 0;
                    
    
    if (scanning_class != cla or confi < confi_threshold):
        gesturePresent = False;
        startTime = 0;
        #print('Bad Input. No Gesture Recognised or Not Clear');
            
            
    if (len(password) > 2):
        break
        
    
    cv2.imshow('output', label)
    
    
    # Stop if escape key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
    #print('running');    
cap.release();
cv2.destroyAllWindows();



         xmin      ymin         xmax        ymax  confidence  class       name
0  668.489563  24.15873  1362.958252  753.409241     0.36708      1  Mini Love
         xmin      ymin         xmax         ymax  confidence  class  \
0  659.341736  9.410339  1390.486938  1032.659424    0.302573      1   

        name  
0  Mini Love  
         xmin       ymin         xmax         ymax  confidence  class  \
0  590.341614  95.045563  1532.320679  1051.341064      0.4547      1   

        name  
0  Mini Love  
         xmin       ymin         xmax         ymax  confidence  class  \
0  622.948242  99.116318  1500.738037  1047.886719    0.370744      1   

        name  
0  Mini Love  
         xmin        ymin        xmax         ymax  confidence  class  \
0  667.781982  108.465363  1462.41626  1047.134521    0.510296      1   

        name  
0  Mini Love  
         xmin        ymin         xmax         ymax  confidence  class  \
0  644.897278  111.433456  1487.397339  1052.521851     0.5235

          xmin       ymin    xmax         ymax  confidence  class       name
0  1101.390015  12.399216  1920.0  1022.677551    0.540104      2  Telephone
          xmin        ymin         xmax         ymax  confidence  class  \
0  1078.967041  154.266815  1920.000000  1043.069580    0.356473      2   
1   619.608337  202.508698  1784.593994  1045.744263    0.264773      0   

        name  
0  Telephone  
1        Kon  
          xmin        ymin         xmax         ymax  confidence  class  \
0  1078.967041  154.266815  1920.000000  1043.069580    0.356473      2   
1   619.608337  202.508698  1784.593994  1045.744263    0.264773      0   

        name  
0  Telephone  
1        Kon  
          xmin       ymin         xmax         ymax  confidence  class  \
0  1091.023682  59.811951  1918.532593  1010.256287    0.425591      2   

        name  
0  Telephone  
          xmin        ymin         xmax         ymax  confidence  class  \
0  1102.983154  205.858749  1920.000000  1032.4702

         xmin        ymin         xmax        ymax  confidence  class  \
0  651.773132  127.138733  1693.885986  751.044067    0.507365      1   

        name  
0  Mini Love  
         xmin        ymin         xmax        ymax  confidence  class  \
0  650.241577  109.420967  1729.496338  856.120483    0.569956      1   

        name  
0  Mini Love  
         xmin        ymin         xmax        ymax  confidence  class  \
0  717.513794  122.617561  1831.880127  847.932373    0.633492      1   

        name  
0  Mini Love  
        xmin        ymin        xmax        ymax  confidence  class       name
0  744.85376  130.341202  1805.06958  866.840881    0.625101      1  Mini Love
         xmin        ymin         xmax        ymax  confidence  class  \
0  787.079041  134.465057  1735.868774  813.469116    0.705701      1   

        name  
0  Mini Love  
         xmin        ymin         xmax       ymax  confidence  class  \
0  772.268372  136.624298  1760.615845  849.18512    0.731529 

         xmin        ymin         xmax        ymax  confidence  class  \
0  683.292236  124.415428  1298.284058  609.878174    0.483132      1   

        name  
0  Mini Love  
        xmin      ymin         xmax        ymax  confidence  class       name
0  824.12915  81.49157  1463.752808  578.942505    0.293416      1  Mini Love
          xmin        ymin         xmax         ymax  confidence  class name
0  1406.332153  380.882538  1844.745239  1056.722046    0.485274      0  Kon
         xmin       ymin         xmax       ymax  confidence  class       name
0  724.203369  72.810745  1413.236572  598.27948    0.420495      1  Mini Love
        xmin       ymin        xmax        ymax  confidence  class       name
0  850.34729  97.979874  1778.89502  741.410706    0.300603      1  Mini Love
         xmin       ymin         xmax        ymax  confidence  class  \
0  830.046021  87.248245  1801.020752  899.902588    0.323314      1   

        name  
0  Mini Love  
         xmin       ymin

         xmin       ymin         xmax        ymax  confidence  class  \
0  793.504761  85.626755  1708.746094  864.977234     0.66202      1   

        name  
0  Mini Love  
        xmin       ymin         xmax        ymax  confidence  class       name
0  799.78241  95.055565  1692.533936  777.256775    0.552925      1  Mini Love
          xmin      ymin         xmax    ymax  confidence  class       name
0  1236.476929  86.06694  1864.447021  1080.0    0.701313      1  Mini Love
          xmin       ymin         xmax    ymax  confidence  class       name
0  1253.595459  88.993561  1880.346069  1080.0    0.719048      1  Mini Love
        xmin       ymin         xmax    ymax  confidence  class       name
0  1139.9198  77.566086  1897.984253  1080.0    0.403232      1  Mini Love
          xmin       ymin         xmax    ymax  confidence  class       name
0  1225.618042  57.935944  1879.943970  1080.0    0.666841      1  Mini Love
1    28.918579  46.831787   248.402939  1080.0    0.34264

         xmin       ymin         xmax    ymax  confidence  class       name
0  1196.64917  72.351013  1895.341553  1080.0    0.788229      1  Mini Love
          xmin       ymin         xmax    ymax  confidence  class       name
0  1183.214722  66.153351  1896.103638  1080.0    0.772592      1  Mini Love
          xmin       ymin         xmax    ymax  confidence  class       name
0  1195.699951  64.021957  1891.665161  1080.0    0.729644      1  Mini Love
          xmin      ymin         xmax    ymax  confidence  class       name
0  1184.006226  70.79155  1901.523682  1080.0    0.809181      1  Mini Love
          xmin        ymin         xmax         ymax  confidence  class  \
0  1186.565552   72.541992  1898.322876  1080.000000    0.800439      1   
1   618.442932  147.774536  1321.181396   685.847168    0.258509      1   

        name  
0  Mini Love  
1  Mini Love  
          xmin        ymin         xmax         ymax  confidence  class  \
0  1186.565552   72.541992  1898.322876  1

         xmin        ymin         xmax        ymax  confidence  class  \
0  636.975464  167.683365  1513.123779  1068.26709    0.343718      1   

        name  
0  Mini Love  
         xmin        ymin         xmax         ymax  confidence  class  \
0  679.306946  175.737579  1508.234009  1057.735718    0.379346      1   

        name  
0  Mini Love  
         xmin        ymin        xmax         ymax  confidence  class  \
0  666.294434  175.380066  1514.42395  1043.627563     0.36565      1   

        name  
0  Mini Love  
         xmin        ymin         xmax         ymax  confidence  class  \
0  764.951904  148.617828  1396.463989  1057.127808    0.624355      1   

        name  
0  Mini Love  
         xmin        ymin         xmax         ymax  confidence  class  \
0  773.579956  145.542114  1373.497314  1063.276489     0.67212      1   

        name  
0  Mini Love  
         xmin       ymin         xmax         ymax  confidence  class  \
0  713.337402  144.35582  1406.72497

         xmin        ymin         xmax         ymax  confidence  class  \
0  597.439453  135.577469  1521.996094  1025.838623    0.329937      1   

        name  
0  Mini Love  
         xmin        ymin         xmax        ymax  confidence  class  \
0  564.665588  151.216278  1410.912964  986.349792    0.279051      1   

        name  
0  Mini Love  
         xmin       ymin         xmax        ymax  confidence  class  \
0  547.460083  90.707428  1263.792847  956.788208    0.250312      1   

        name  
0  Mini Love  


In [297]:
cap.release();
cv2.destroyAllWindows();
